In [2]:
%matplotlib inline

In [3]:
import torch
import time
import torchvision
import torchvision.transforms as transforms
import copy
import cv2
import numpy as np
import pickle
import os
from PIL import Image, ImageChops
import random
from matplotlib import pyplot as plt
import shutil
from shutil import copy2
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
#下载图片

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#将 cifar 里的所有数据扒出来保存为图片文件

# 解压缩，返回解压后的字典
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def cifar10_to_images():
    tar_dir='./cifar-10-batches-py/' #原始数据库目录
    train_root_dir='./cifar10/train/' #图片保存目录
    test_root_dir='./cifar10/test/'
    if not os.path.exists(train_root_dir):
        os.makedirs(train_root_dir)
    if not os.path.exists(test_root_dir):
        os.makedirs(test_root_dir)
    # 生成训练集图片，如果需要png格式，只需要改图片后缀名即可。
    label_names = ["airplane", "automobile", "bird", "cat", "dear", "dog", "frog", "horse", "ship", "truck"]
    for j in range(1, 6):
        dataName = tar_dir+"data_batch_" + str(j) 
        Xtr = unpickle(dataName)
        print(dataName + " is loading...")

        for i in range(0, 10000):
            img = np.reshape(Xtr['data'][i], (3, 32, 32))  # Xtr['data']为图片二进制数据
            img = img.transpose(1, 2, 0)  # 读取image
            picName = train_root_dir + str(Xtr['labels'][i])+ '_' + label_names[Xtr['labels'][i]]+'_'+ str(i + (j - 1)*10000)+'.jpg'  # label+class+index
            cv2.imwrite(picName, img)
        print(dataName + " loaded.")

    print("test_batch is loading...")
 
    # 生成测试集图片
    testXtr = unpickle(tar_dir+"test_batch")
    for i in range(0, 10000):
        img = np.reshape(testXtr['data'][i], (3, 32, 32))
        img = img.transpose(1, 2, 0)
        picName = test_root_dir + str(testXtr['labels'][i])+ '_' + label_names[testXtr['labels'][i]]+'_' + str(i) + '.jpg'
        cv2.imwrite(picName, img)
    print("test_batch loaded.")
    
cifar10_to_images()

./cifar-10-batches-py/data_batch_1 is loading...
./cifar-10-batches-py/data_batch_1 loaded.
./cifar-10-batches-py/data_batch_2 is loading...
./cifar-10-batches-py/data_batch_2 loaded.
./cifar-10-batches-py/data_batch_3 is loading...
./cifar-10-batches-py/data_batch_3 loaded.
./cifar-10-batches-py/data_batch_4 is loading...
./cifar-10-batches-py/data_batch_4 loaded.
./cifar-10-batches-py/data_batch_5 is loading...
./cifar-10-batches-py/data_batch_5 loaded.
test_batch is loading...
test_batch loaded.


In [6]:
#读取并且反转

def read_directory(directory_name):
# this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #读取图片
        fname = filename
        img = Image.open('./data/' + filename)
        img_inverted = ImageChops.invert(img)
        img_inverted.save('./data_convert/' + filename)

read_directory('./data')

In [7]:
#加高斯模糊

kernel_size = (3, 3)
sigma = 1.5

def read_directory(directory_name):
# this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #读取图片
        
        imgName = filename
        img = cv2.imread('./data_convert/'+imgName)
        img = cv2.GaussianBlur(img, kernel_size, sigma)
        new_imgName = imgName
        cv2.imwrite('./data_convert_mohu/'+new_imgName, img)

read_directory('./data_convert')

In [8]:
#加满足高斯分布的白噪声

def gasuss_noise(image, mean=0, var=0.001):
 
    '''
        添加高斯噪声
        mean : 均值
        var : 方差
    '''
 
    image = np.array(image/255, dtype=float)
    noise = np.random.normal(mean, var ** 0.5, image.shape)
    out = image + noise
 
    '''
        将值限制在(-1/0,1)间，然后乘255恢复
    '''

    if out.min() < 0:
        low_clip = -1.
    else:
        low_clip = 0.
 
    out = np.clip(out, low_clip, 1.0)
    out = np.uint8(out*255)
    return out

def read_directory(directory_name):
# this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #读取图片
        imgName = filename
        img = cv2.imread('./data_convert_mohu/'+imgName)
        # 添加高斯噪声，均值为0，方差为0.001
        out2 = gasuss_noise(img, mean=0, var=0.001)
        new_imgName = imgName
        cv2.imwrite('./data_convert_mohu_baizao/'+new_imgName, out2)

read_directory('./data_convert_mohu')

In [9]:
#求 x 和 y 方向图像梯度

# Scharr算子实现梯度计算
def Scharr_demo(image):
    # x 方向梯度
    image_grad_x = cv2.Scharr(image, cv2.CV_32F, 1, 0)
    # y 方向梯度
    image_grad_y = cv2.Scharr(image, cv2.CV_32F, 0, 1)
    # 分别求绝对值并转化为8位的图像上，这样做方便显示
    image_gradx = cv2.convertScaleAbs(image_grad_x) 
    image_grady = cv2.convertScaleAbs(image_grad_y)
#     # 显示两个方向图像
#     cv2.imshow("image_gradient-x", image_gradx)
#     cv2.imshow("image_gradient-y", image_grady)
    #两个方向梯度的叠加，权重各自一半
    image_gradxy = cv2.addWeighted(image_gradx, 0.5, image_grady, 0.5, 0)
#     cv2.imshow("image_gradient", image_gradxy)
    return image_gradxy

def read_directory(directory_name):
# this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #读取图片
        imgName = filename
        img = cv2.imread('./data_convert_mohu_baizao/'+imgName)
        out = Scharr_demo(img)
        new_imgName = imgName
        cv2.imwrite('./data_convert_mohu_baizao_tidu/'+new_imgName, out)

read_directory('./data_convert_mohu_baizao')

In [10]:
#将扒出来的图片按 8:1:1 分成训练集，验证集和测试集。
#分别保存在 train，validation， test 三个 folder 中。

datadir_normal = "./data_convert_mohu_baizao_tidu/"

all_data = os.listdir(datadir_normal)#（图片文件夹）
num_all_data = len(all_data)
print( "num_all_data: " + str(num_all_data) )
index_list = list(range(num_all_data))
#print(index_list)
random.shuffle(index_list)
num = 0

trainDir = "./811/train/"#（将训练集放在这个文件夹下）
if not os.path.exists(trainDir):
    os.mkdir(trainDir)
        
validDir = './811/validation/'#（将验证集放在这个文件夹下）
if not os.path.exists(validDir):
    os.mkdir(validDir)
        
testDir = './811/test/'#（将测试集放在这个文件夹下）        
if not os.path.exists(testDir):
    os.mkdir(testDir)
        
for i in index_list:
    fileName = os.path.join(datadir_normal, all_data[i])
    if num < num_all_data*0.8:
        #print(str(fileName))
        copy2(fileName, trainDir)
    elif num>num_all_data*0.8 and num < num_all_data*0.9:
        #print(str(fileName))
        copy2(fileName, validDir)
    else:
        copy2(fileName, testDir)
    num += 1

num_all_data: 58994


In [11]:
# img = cv2.imread('0_airplane_3_反转_高斯模糊_高斯白噪.jpg')
# # x 方向梯度
# image_grad_x = cv2.Scharr(img, cv2.CV_32F, 1, 0)
# # y 方向梯度
# image_grad_y = cv2.Scharr(img, cv2.CV_32F, 0, 1)
# # 分别求绝对值并转化为8位的图像上，这样做方便显示
# image_gradx = cv2.convertScaleAbs(image_grad_x) 
# image_grady = cv2.convertScaleAbs(image_grad_y)
# image_gradxy = cv2.addWeighted(image_gradx, 0.5, image_grady, 0.5, 0)
# cv2.imwrite('./x.jpg', image_gradx)
# cv2.imwrite('./y.jpg', image_grady)
# cv2.imwrite('./xy.jpg', image_gradxy)